In [2]:
import os
import glob
import logging
import concurrent.futures
from tqdm import tqdm

from sturdy_barnacle.db_utils import DatabaseManager
from sturdy_barnacle.image_processor import ImageProcessor

num_workers = max(os.cpu_count() - 4, 1)


logging.basicConfig(
    filename="image_processing.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Initialize DatabaseManager
db = DatabaseManager()

def process_image(image_path: str) -> None:
    """
    Process an image if it hasn’t been processed. 
    Skips it if already processed and logs any errors.
    """
    try:
        if db.is_image_processed(image_path):
            logging.info(f"Skipping {image_path}, already processed.")
            return

        processor = ImageProcessor(image_path, db)
        processor.process_and_store()

    except Exception as e:
        logging.error(f"Error processing {image_path}: {e}")


IMAGE_DIR = "/Users/mamduhzabidi/Pictures/Photos Library.photoslibrary/originals/0/"
image_paths = glob.glob(f"{IMAGE_DIR}/*.jpeg")

image_paths = image_paths[:10]

with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    with tqdm(total=len(image_paths), desc="Processing Images") as pbar:
        futures = {executor.submit(process_image, img): img for img in image_paths}
        for future in concurrent.futures.as_completed(futures):
            pbar.update(1)

Processing Images:   0%|                                                                                                                                                                                               | 0/10 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/mamduhzabidi/Documents/GitHub/sturdy-barnacle/myenv/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Processing Images: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:53<00:00, 11.35s/it]
